In [3]:
import warnings
warnings.filterwarnings('ignore')

import grpc
import tensorflow as tf
from keras_image_helper import create_preprocessor
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [ ]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape = data.shape)

In [18]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

preprocessor =  create_preprocessor('xception', target_size= (299,299))

In [19]:
url  = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)


pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [22]:
pb_response = stub.Predict(pb_request, timeout= 20.0)
pred = pb_response.outputs['dense_7'].float_val

classes = ['dress',
            'hat',
            'longsleeve',
            'outwear',
            'pants',
            'shirt',
            'shoes',
            'shorts',
            'skirt',
            't-shirt']

out = dict(zip(classes, pred))